<a href="https://colab.research.google.com/github/toan-ly/VinaLLaMA-Math-Solver/blob/main/vinallama_math_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from pprint import pprint
from tqdm import tqdm
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Sign in to hugging face

In [ ]:
notebook_login()

# Load the pretrained LLM

In [ ]:
MODEL_NAME = 'vilm/vinallama-7b-chat'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'up_proj',
        'o_proj',
        'k_proj',
        'down_proj',
        'gate_proj',
        'v_proj'
    ],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f'Trainable params: {trainable_params} || All params: {all_params} || Trainables%: {100 * trainable_params / all_params}')

In [ ]:
print_trainable_parameters(model)

Trainable params: 39976960 || All params: 3657576448 || Trainables%: 1.092990415056391


# Test pretrained model performance

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
prompt = '''
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 2 chục đọc là:
### Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
### Các câu trả lời:

<|im_start|>assistant
'''.strip()

In [ ]:
def predict_result(prompt):
    encoding = tokenizer(prompt, return_tensors='pt').to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
%%time
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

predict_result(prompt)

<IPython.core.display.Javascript object>

<|im_start|> system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|> user
### Câu hỏi:
Số gồm 1 đơn vị và 2 chục đọc là:
### Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
### Các câu trả lời:

<|im_start|> assistant
Để giải quyết vấn đề này, chúng ta cần hiểu rằng "1 đơn vị" có nghĩa là một đơn vị (một đơn vị riêng lẻ), và "2 chục" có nghĩa là hai nhóm mười.

Chúng ta có thể bắt đầu bằng cách viết số dưới dạng phân số của một đơn vị và hai chục.

1 đơn vị + 2 chục = 1 + 2x (trong đó x đại diện cho số lượng mười)

Chúng ta biết rằng số đọc là 30, vì vậy chúng ta có thể thiết lập phương trình:

30 = 1 + 2x

Bây giờ, chúng ta có thể giải phương trình để tìm x:

30 - 1 = 2x
29 = 2x

Bây giờ, chúng ta có thể giải phương trình để tìm x:

x = 29 / 2
x = 14,5

Vì chúng ta không thể có một nửa chục, chúng ta làm tròn lên đến số nguyên tiếp theo, đó
CPU times: user 32.1 s, sys: 1.11 s, total: 33.2 s
W

# Fine-tuning LLM
## Download Dataset

In [ ]:
data = load_dataset('hllj/vi_grade_school_math_mcq')
data

Generating train split:   0%|          | 0/2733 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['grade', 'id', 'title', 'problems', 'url'],
        num_rows: 2733
    })
})

In [ ]:
type(data)

datasets.dataset_dict.DatasetDict

In [ ]:
data['train']

Dataset({
    features: ['grade', 'id', 'title', 'problems', 'url'],
    num_rows: 2733
})

In [ ]:
data['train'][5]['problems'][0]

{'choices': [],
 'explanation': 'Lời giải: \n a. 2 + 3 = 5          \n 3 + 3 = 6        \n 1 + 4 = 5        \n 2 + 4 = 6 \n b. 1 + 4 + 3 = 8                        \n 2 + 3 + 3 = 8',
 'question': 'Câu 1: \n \n \n Tính: \n a)  \n 2 + 3 = …          \n 3 + 3 = …              \n 1 + 4 = …              \n 2 + 4 = … \n b) 1 + 4 + 3 = …'}

## Prepare training dataset

In [ ]:
def generate_prompt(question, choices, explaination):
    return f'''
    <|im_start|>system
    Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

    <|im_start|>user
    ### Câu hỏi:
    {question}
    ### Các lựa chọn:
    {choices}
    ### Câu trả lời:

    <|im_start|>assistant
    {explaination}
    '''.strip()

def generate_and_tokenize_prompt(question, choices, explaination):
    full_prompt = generate_prompt(question, choices, explaination)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [ ]:
training_samples = []
for sample in tqdm(data['train']):
    for question in sample['problems']:
        choices = question['choices']
        explaination = question['explanation'].strip()
        question = question['question']

        if explaination == '' or question == '' or choices == []:
            continue

        try:
            question = question.split('\n \n')[1].strip()
        except:
            continue

        choices = '\n'.join(choices)
        training_sample = generate_and_tokenize_prompt(question, choices, explaination)
        training_samples.append(training_sample)

100%|██████████| 2733/2733 [00:04<00:00, 623.73it/s]


In [ ]:
data_train = Dataset.from_list(training_samples)

# Training

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir='experiments',
    optim='paged_adamw_8bit',
    lr_scheduler_type='cosine',
    warmup_ratio=0.05
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data_train,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.468400
2,4.072100
3,4.243100
4,4.005000
5,4.157000
6,4.102300
7,3.982900
8,3.754200
9,3.809500
10,3.507300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

Step,Training Loss
1,3.468400
2,4.072100
3,4.243100
4,4.005000
5,4.157000
6,4.102300
7,3.982900
8,3.754200
9,3.809500
10,3.507300


KeyboardInterrupt: 

# Test Prediction

In [ ]:
prompt = """
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 2 chục đọc là :
### Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
### Câu trả lời:

<|im_start|>assistant
""".strip()

predict_result(prompt)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


# Save model to huggingface

In [ ]:
# !huggingface-cli login

In [ ]:
# !pip install huggingface_hub
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
model.save_pretrained('trained-model')

In [ ]:
PEFT_MODEL = 'toan-ly/vinallama-peft-7b-math-solver'

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

# Inference

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
Câu hỏi:
Số gồm 1 đơn vị và 3 chục đọc là :
Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
Câu trả lời:

<|im_start|>assistant
""".strip()

predict_result(prompt)